In [18]:
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io.wavfile as wv
import scipy.signal as sig
import wave
from util import util
from util import WavFileParts
from util.logUtil import LOG, LOG_HEADER
from util.confusionMatrix import ConfusionMatrix

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir
if osDir == winDir:
    storageFolder = 'E:/SAA_DATA/storedData/'
else:
    storageFolder = '/Users/etto/Desktop/storedData/'

Using TensorFlow backend.


In [10]:
def retrieveTestDataMFCC(storageName, keyName):
    filename = storageFolder + storageName + '_MFCC.hd5'
    allMfccsDf = pd.read_hdf(path_or_buf=filename, key='mfccs_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return allMfccsDf.values, classesDf.values

def retrieveTestData(storageName, NFFT, keyName):
    if NFFT == 1024:
        filename = storageFolder + storageName + '.hd5'
    else:
        filename = storageFolder + storageName + '_NFFT{:d}.hd5'.format(NFFT)
    specDf = pd.read_hdf(path_or_buf=filename, key='spectros_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return specDf.values, classesDf.values

In [14]:
testFileName = 'testData_G428'
keyname = 'mic4'
NFFT = 1024
testSpecs, testClasses = retrieveTestData(testFileName, NFFT, keyname)
print(np.shape(testSpecs))

(48300, 513)


In [19]:
testFileName = 'testData_G428'
keyname = 'mic4'
NFFT = 2048
testSpecs, testClasses = retrieveTestData(testFileName, NFFT, keyname)
print(np.shape(testSpecs))

(5913, 1025)


In [16]:
testFileName = 'testData_G428'
keyname = 'mic4'
NFFT = 4096
testSpecs, testClasses = retrieveTestData(testFileName, NFFT, keyname)
print(np.shape(testSpecs))

(48300, 2049)


In [17]:
testFileName = 'testData_G428_MFCC'
keyname = 'mic4'
testSpecs, testClasses = retrieveTestDataMFCC(testFileName, keyname)
print(np.shape(testSpecs))

(7051070, 13)


In [ ]:
def getFilesPerCatFromMultipleDirs(srcDirs, srcDirsBase=''):
    filesPerCat = {}
    for dirName in srcDirs:
        srcDir = srcDirsBase + '/' + dirName
        fpcNw = getFilesPerCategory(srcDir)
        if not filesPerCat:
            filesPerCat = fpcNw
        else:
            for key in filesPerCat:
                filesPerCat[key] += fpcNw[key]
    return filesPerCat

def createTrainDataFromSpectros(spectrosPerCat, clzLengths):
    X_train = np.concatenate(spectrosPerCat[classes[0]], axis=0)
    for i in range(1, len(classes)):
        nwSpectros = np.concatenate(spectrosPerCat[classes[i]], axis=0)
        X_train = np.concatenate((X_train,nwSpectros), axis=0)
    
    # one-hot encoding voor Y_train
    nrFiles = clzLengths[classes[0]]
    Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

    nrFiles = clzLengths[classes[1]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.ones(nrFiles), np.zeros(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

    nrFiles = clzLengths[classes[2]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.zeros(nrFiles), np.ones(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
    
    return X_train, Y_train

# Maakt een dictionary aan; per categorie de spectrogrammen
def getSpectrosFromFilesPerCategory(filesPerCategory, NFFT):
    spectros = {}
    for clz in classes:
        spectros[clz] = []
        for filename in filesPerCategory[clz]:
            fs, signal = wv.read(filename)
            freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
            spectros[clz].append(spectrogram.T)
    return spectros

def getTrainDataFromFolders(orgWavDirs, NFFT):
    fpc = getFilesPerCatFromMultipleDirs(orgWavDirs, baseSrcDir)
    spcs = getSpectrosFromFilesPerCategory(fpc, NFFT)
    clzLengths = getClassLengths(spcs)
    X_train, Y_train = createTrainDataFromSpectros(spcs, clzLengths)
    # real train classes
    realTrainClasses = np.concatenate((np.zeros(clzLengths[classes[0]]), 
                                  np.ones(clzLengths[classes[1]]), 
                                  2*np.ones(clzLengths[classes[2]])))
    return X_train, Y_train, realTrainClasses

In [ ]:
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs, NFFT)